In [2]:
import pandas as pd
import numpy as np
from itertools import chain

In [ ]:
df_vote = pd.read_csv('H117_votes.csv')


In [43]:
#read node file
place_node = pd.read_csv('node district.csv')
# Merge df1 and df2 on 'icprs' column
merged_df_vote = pd.merge(df_vote, place_node[['icpsr']], on='icpsr', how='inner')

In [44]:
# distinct number of icpsr 457, 996 roll
icpsr = df_vote.icpsr.unique()
print(len(icpsr))
icpsr = merged_df_vote.icpsr.unique()
print(len(icpsr))
df_vote = merged_df_vote

69
68


In [45]:
# reshape df
re_df = df_vote.pivot(index='icpsr', columns='rollnumber', values='cast_code')
re_df.to_csv('widevote10_29.csv')

In [47]:
#set threthold for assign an egde:
threshold = 450
edgelist = []
for index, row in re_df.iterrows():
    # print (int(index))
    ex_df = re_df[index:]
    ex_df = ex_df.iloc[1:, :]
    # print(ex_df.shape)
    nrow = row.to_numpy()
    for exi, exr in ex_df.iterrows():
        nexr = exr.to_numpy()
        equal_pos = np.where(nrow == nexr)
        if(len(equal_pos[0]) > threshold ):
            edgelist.append([index,exi])

print(len(edgelist))

1279


In [48]:
# remove value not in the district node file 
newedgelist = list(chain.from_iterable(edgelist))
ic = df_vote.icpsr.unique()
differ = list(set(newedgelist)- set(ic) )
print(differ)
filter_edgelist = []
for edge in edgelist:
    if any(x in differ for x in edge):
        continue
    else: 
        filter_edgelist.append(edge)

print(len(edgelist))
print(len(filter_edgelist))
print(set(chain.from_iterable(filter_edgelist))- set(ic))

[]
1279
1279
set()


In [ ]:
# save edge assign resutls 
result_df = pd.DataFrame(filter_edgelist, columns =['Source', 'Target'])
result_df = result_df.astype('Int64')
filename = 'vote_edge_filter'+ str(threshold) +'.csv'
result_df.to_csv(filename, index=False)